In [ ]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 16)
#pd.set_option('display.width', 2000)
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn import metrics

from functions_folder.nn_MIMICDataset import MIMICDataset
from functions_folder.nn_model_simplex import *
from functions_folder.nn_train import train
from functions_folder.nn_evaluate import evaluate

import os
import pickle as pkl
import sys
import random
import time
import math

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, Dataset


from simplexai.explainers.simplex import Simplex
import os
import pickle as pkl
import sys
import random

# Remove printing error
pd.options.mode.chained_assignment = None

In [2]:
# Data
mimic_data = pd.read_csv('mimic_data.csv')


# Split into 'preprocessing dataset' and a 'train/vaid/test dataset'
stays = mimic_data['stay_id'].unique()
import random
random.Random(0).shuffle(stays)
model_data2 = mimic_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.5 * len(stays))
preprocessing_stays = stays[:n]
hold_out_stays = stays[n:]
preprocessing_data = model_data2[model_data2['stay_id'].isin(preprocessing_stays)]
hold_out_data = model_data2[model_data2['stay_id'].isin(hold_out_stays)]
#preprocessing_data = hold_out_data
preprocessing_data.drop(columns=['iv_treatment_length'], inplace=True)
preprocessing_data

stay_id        date  iv_flag  first_po_flag  po_flag  \
0      39409066.0  2137-05-22      1.0            1.0        0   
1      39409066.0  2137-05-23      1.0            1.0        0   
2      30322661.0  2169-10-09      1.0            NaN        0   
3      30322661.0  2169-10-10      1.0            1.0        0   
4      30322661.0  2169-10-11      1.0            1.0        0   
...           ...         ...      ...            ...      ...   
14515  37575956.0  2153-02-15      1.0            1.0        0   
14516  37575956.0  2153-02-16      1.0            1.0        0   
14517  37540542.0  2161-05-10      1.0            NaN        0   
14518  37540542.0  2161-05-11      1.0            1.0        0   
14519  37540542.0  2161-05-12      1.0            1.0        0   

       blood pressure diastolic0  blood pressure diastolic1  \
0                       0.575521                   0.519798   
1                       0.448568                   0.454810   
2                       0.432292                   0.446094   
3                       0.361979                   0.375081   
4                       0.397135                   0.378309   
...                          ...                        ...   
14515                   0.479167                   0.536475   
14516                   0.522135                   0.525500   
14517                   0.253906                   0.237573   
14518                   0.397135                   0.370562   
14519                   0.389323                   0.422208   

       blood pressure diastolic2  blood pressure diastolic3  \
0                       0.272727                   0.700000   
1                       0.454545                   0.411765   
2                       0.909091                   0.304348   
3                       0.545455                   0.304348   
4                       0.181818                   0.500000   
...                          ...                        ...   
14515                   0.090909                   0.875000   
14516                   0.272727                   1.000000   
14517                   0.090909                  -1.000000   
14518                   0.363636                   0.304348   
14519                   0.272727                   0.411765   

       blood pressure diastolic4  blood pressure diastolic5  \
0                            0.0                        0.0   
1                            0.0                        0.0   
2                            0.0                        0.0   
3                            0.0                        0.0   
4                            0.0                        0.0   
...                          ...                        ...   
14515                        0.0                        0.0   
14516                        0.0                        0.0   
14517                        0.0                        0.0   
14518                        0.0                        0.0   
14519                        0.0                        0.0   

       blood pressure diastolic6  blood pressure diastolic7  \
0                       0.000000               1.968181e-31   
1                       1.000000               1.184155e-01   
2                       0.000000               2.322353e-01   
3                       0.666667               2.231539e-01   
4                       0.333333               1.405844e-31   
...                          ...                        ...   
14515                   0.000000               0.000000e+00   
14516                   0.000000               6.248194e-32   
14517                   0.166667               0.000000e+00   
14518                   0.000000               1.430673e-01   
14519                   0.166667               1.430673e-01   

       blood pressure diastolic8  blood pressure diastolic9  \
0                       0.866667                   0.302869   
1                       0.133333                   0.195795   


In [3]:
all_shap_importance = pd.read_csv('all_shap_importance.csv')

shap_list = all_shap_importance[all_shap_importance['feature_importance_vals'] >= 0.5].col_name.unique().tolist()
shap_list = ['stay_id', 'date', 'iv_flag', 'first_po_flag', 'po_flag'] + shap_list
hold_out_data = hold_out_data[shap_list]
hold_out_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure systolic17_current_stay,heart rate17,respiratory rate11_current_stay,blood pressure mean11_current_stay,o2 saturation pulseoxymetry3_current_stay,gcs - motor response15_current_stay,heart rate17_current_stay,blood pressure systolic11_current_stay,heart rate18_current_stay,temperature5_difference,gcs - verbal response10_current_stay,heart rate14,blood pressure diastolic17_current_stay,o2 saturation pulseoxymetry21,o2 saturation pulseoxymetry15_current_stay,o2 saturation pulseoxymetry17,respiratory rate5,gcs - verbal response16_current_stay,o2 saturation pulseoxymetry5_difference,gcs - motor response10_current_stay,blood pressure systolic19_current_stay,respiratory rate17_current_stay,gcs - verbal response16,blood pressure mean5_current_stay,blood pressure systolic14_current_stay,o2 saturation pulseoxymetry10_difference,gcs - eye opening13_current_stay,o2 saturation pulseoxymetry10,heart rate6_difference,gcs - verbal response_mean_current_stay,blood pressure diastolic10_difference,blood pressure diastolic20_current_stay,respiratory rate6_current_stay_difference,gcs - motor response_std_difference,temperature7_current_stay_difference,blood pressure systolic15_current_stay,heart rate9_current_stay_difference,blood pressure systolic18_current_stay,blood pressure mean6_difference,blood pressure systolic5_difference,respiratory rate15_current_stay_difference,heart rate3,o2 saturation pulseoxymetry9,o2 saturation pulseoxymetry2,gcs - motor response_std_current_stay,o2 saturation pulseoxymetry17_current_stay,gcs - verbal response7_current_stay,temperature20_current_stay,blood pressure systolic0_difference,respiratory rate5_current_stay_difference,o2 saturation pulseoxymetry20_current_stay,blood pressure systolic_mean,o2 saturation pulseoxymetry18_current_stay,gcs - eye opening7_current_stay,gcs - eye opening1,heart rate2_current_stay,gcs - motor response13,heart rate16_current_stay,gcs - motor response6,gcs - motor response20_current_stay,heart rate3_current_stay,gcs - motor response0_current_stay,heart rate16,temperature10_difference,heart rate2_current_stay_difference,respiratory rate15_difference,heart rate21,gcs - motor response15,gcs - motor response_mean,blood pressure mean13_difference,heart rate11_current_stay,gcs - eye opening_std_difference,blood pressure mean12_difference,blood pressure diastolic3_current_stay,gcs - eye opening21,respiratory rate12_difference,blood pressure mean3_current_stay,o2 saturation pulseoxymetry6_current_stay,blood pressure systolic10,blood pressure diastolic12,heart rate20,heart rate13_difference,gcs - motor response8_current_stay,blood pressure systolic12_current_stay,o2 saturation pulseoxymetry1_current_stay_difference,gcs - verbal response14_current_stay,temperature7_current_stay,temperature6_current_stay_difference,respiratory rate3,heart rate21_current_stay,o2 saturation pulseoxymetry7_difference,gcs - verbal response2_difference,heart rate8
14520,31658148.0,2154-01-11,1.0,1.0,0,0.606696,-1.000000,0.836298,7.378308e-01,0.843672,-1.000000,0.808321,0.830774,-1.000000,-1.0,-1.000000,0.4,0.930269,0.000000,0.293968,0.465445,0.0,0.495327,-1.0,-1.000000,-1.000000,0.519153,0.0,0.000000,0.250,-1.000000,1.000000,0.469564,-1.00,0.333333,-1.000000,1.000000,-1.0,-1.000000,-1.000000,0.734471,-1.000000,-1.000000,-1.0,-1.0,-1.000000,0.589744,0.250657,0.333333,-1.0,0.522908,0.0,-1.000000,-1.000000,-1.0,1.000000,0.533294,-1.000000,0.000000e+00,0.544681,0.142857,-1.0,0.233184,0.181818,1.000000,0.796165,0.0,0.120000,-1.000000,-1.000000,-1.000000,0.500000,0.000000,-1.0,-1.000000,0.887405,-1.000000,-1.0,0.725256,0.0,-1.0,0.773994,0.5,0.118421,0.333333,1.000000,-1.000000,0.496063,0.000000,-1.000000,0.0,-1.000000e+00,-1.0,0.974359,0.111111,-1.000000,-1.0,0.076923
14521,31658148.0,2154-01-12,1.0,1.0,0,0.425485,0.626496,0.179521,3.985847e-01,0.505667,0.415793,0.395976,0.268953,0.634286,-1.0,-1.000000,0.4,0.404433,0.666667,0.726911,0.330814,0.0,0.242991,-1.0,-1.0

In [4]:
individual = '110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [5]:
stays = hold_out_data['stay_id'].unique()
import random
random.Random(25).shuffle(stays) # Best split from CV

X_data = hold_out_data.drop(columns=['stay_id', 'po_flag', 'date', 'iv_flag', 'first_po_flag'])
# Filter for features in this individual
X_data = X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
model_data = pd.concat([hold_out_data[['stay_id', 'po_flag']], X_data], axis=1)
model_data2 = model_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.7 * len(stays))
n2 = round(0.85 * len(stays))
train_stays = stays[:n]
validation_stays = stays[n:n2]
test_stays = stays[n2:]
train_data = model_data2[model_data2['stay_id'].isin(train_stays)]
valid_data = model_data2[model_data2['stay_id'].isin(validation_stays)]
test_data = model_data2[model_data2['stay_id'].isin(test_stays)]

In [19]:
# Set the random seeds for deterministic results.
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

# Define batch size 
batch_size = 256

# Define dataloaders
train_dataset = MIMICDataset(train_data)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn=train_dataset.collate_fn_padd)

valid_dataset = MIMICDataset(valid_data)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=batch_size, collate_fn=valid_dataset.collate_fn_padd)

test_dataset = MIMICDataset(test_data)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, collate_fn=test_dataset.collate_fn_padd)

# Define dual_catagorial
dual_catagorial = False

# Hyperparameters
INPUT_DIM = 5
OUTPUT_DIM = 1
HID_DIM = 512 
HID_DIM2 = 16 
DROPOUT = 0.4

# Define model
model = Model_simplex_short(INPUT_DIM, OUTPUT_DIM, HID_DIM, HID_DIM2, DROPOUT).to(device)

# Calculate the number of trainable parameters in the model.

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# Define optimizer and learning_rate
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define loss
criterion = nn.BCEWithLogitsLoss()

GPU is available
The model has 11,307 trainable parameters


In [7]:
# -----------------------------
# Evaluate best model on test set
# -----------------------------

model.load_state_dict(torch.load('hold_out_switch_model_short.pt'))

test_loss, test_accuracy, test_auroc, test_predictions, test_labels = evaluate(model, test_dataloader, criterion)

print(f'\n| Test Loss: {test_loss} | Test PPL: {math.exp(test_loss)} |')
print('Test accuracy result:', test_accuracy)
print('Test AUROC result:', test_auroc)
#torch.set_printoptions(edgeitems=10)
print(np.unique(test_predictions.round(), return_counts=True))
print(np.unique(test_labels.round(), return_counts=True))
print('test_predictions', test_predictions)
print('test_labels', test_labels)

<All keys matched successfully>

100%|██████████| 9/9 [00:02<00:00,  3.94it/s]


| Test Loss: 0.4391332467397054 | Test PPL: 1.5513619875432905 |
Test accuracy result: 0.7432950191570882
Test AUROC result: 0.8072253826755029
(array([0., 1.], dtype=float32), array([1266,  822]))
(array([0., 1.], dtype=float32), array([1734,  354]))
test_predictions [8.6060417e-04 6.3692987e-02 3.1436728e-05 ... 8.2787983e-02 5.2258557e-01
 6.3019961e-01]
test_labels [0. 0. 0. ... 0. 0. 0.]


In [8]:
def new_threshold_fun(predictions, lower_bound=0.5427614, upper_bound=0.736):
    lower_bound_predictions = [1 if a_ >= lower_bound else 0 for a_ in predictions]
    upper_bound_predictions = [1 if a_ >= upper_bound else 0 for a_ in predictions]
    return lower_bound_predictions, upper_bound_predictions

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score

In [10]:
# Use new cut off
lower_bound_test_predictions, upper_bound_test_predictions = new_threshold_fun(test_predictions)

# Lower bound
roc_auc_score(test_labels, lower_bound_test_predictions)
accuracy_score(test_labels, lower_bound_test_predictions)
balanced_accuracy_score(test_labels, lower_bound_test_predictions)
recall_score(test_labels, lower_bound_test_predictions)
precision_score(test_labels, lower_bound_test_predictions)
f1_score(test_labels, lower_bound_test_predictions)
average_precision_score(test_labels, lower_bound_test_predictions)
test_cm = confusion_matrix(test_labels, lower_bound_test_predictions)
test_true_positive_rate = (test_cm[0][0]) / (test_cm[0][0] + test_cm[0][1])
test_false_positive_rate = (test_cm[1][0]) / (test_cm[1][0] + test_cm[1][1])
test_cm
test_true_positive_rate
test_false_positive_rate

# Upper bound
roc_auc_score(test_labels, upper_bound_test_predictions)
accuracy_score(test_labels, upper_bound_test_predictions)
balanced_accuracy_score(test_labels, upper_bound_test_predictions)
recall_score(test_labels, upper_bound_test_predictions)
precision_score(test_labels, upper_bound_test_predictions)
f1_score(test_labels, upper_bound_test_predictions)
average_precision_score(test_labels, upper_bound_test_predictions)
ub_test_cm = confusion_matrix(test_labels, upper_bound_test_predictions)
ub_test_true_positive_rate = (ub_test_cm[0][0]) / (ub_test_cm[0][0] + ub_test_cm[0][1])
ub_test_false_positive_rate = (ub_test_cm[1][0]) / (ub_test_cm[1][0] + ub_test_cm[1][1])
ub_test_cm
ub_test_true_positive_rate
ub_test_false_positive_rate

0.8077776474498074

0.7610153256704981

0.8077776474498074

0.8785310734463276

0.40547588005215124

...

0.5225225225225225

0.3475180183478802

array([[1514,  220],
       [ 151,  203]])

0.8731257208765859

0.4265536723163842

### Simplex

In [8]:
# Split X y
X_train_data = train_data.drop(columns=['stay_id', 'po_flag'])
y_train_data = train_data['po_flag']
X_test_data = test_data.drop(columns=['stay_id', 'po_flag'])
y_test_data = test_data['po_flag']

In [9]:
corpus_size=len(X_train_data)

In [10]:
class TestDataset(Dataset):
    def __init__(self, X, y=None) -> None:
        self.X = X
        self.y = y.astype(int)

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i: int) -> tuple:
        data = torch.tensor(self.X.iloc[i, :], dtype=torch.float32)
        target = self.y.iloc[i]
        return data, target

In [11]:
train_data_corpus = TestDataset(X_train_data, y_train_data)
corpus_loader = DataLoader(train_data_corpus, batch_size=corpus_size, shuffle=False)
corpus_examples = enumerate(corpus_loader)
batch_id_corpus, (corpus_inputs, corpus_targets) = next(corpus_examples)

In [12]:
batch_size = 256
test_data_corpus = TestDataset(X_test_data, y_test_data)
test_loader = DataLoader(test_data_corpus, batch_size=batch_size, shuffle=False)
test_examples = enumerate(test_loader)
batch_id_test, (test_inputs, test_targets) = next(test_examples)

In [22]:
input_baseline = torch.full(corpus_inputs.shape, torch.mean(corpus_inputs)) # Baseline tensor of the same shape as corpus_inputs

In [13]:
# sort order function for decomposition
def apply_sort_order(in_list, sort_order):
    if isinstance(in_list, list):
        return [in_list[idx] for idx in sort_order]
    if torch.is_tensor(in_list):
        return [in_list.numpy()[idx] for idx in sort_order]

In [14]:
def load_trained_model(model, trained_model_state_path):
    model.load_state_dict(torch.load(trained_model_state_path))
    # model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    model.eval()
    return model

In [20]:
# Load model
load_trained_model(model, 'VSCode_projects/stop/python_scripts/hold_out_switch_model_short.pt')

Model_simplex_short(
  (layers): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=512, out_features=16, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
  )
  (layers2): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [15]:
def new_threshold_fun(predictions, lower_bound=0.5427614, upper_bound=0.736):
    lower_bound_predictions = [1 if a_ >= lower_bound else 0 for a_ in predictions]
    upper_bound_predictions = [1 if a_ >= upper_bound else 0 for a_ in predictions]
    return lower_bound_predictions, upper_bound_predictions

In [19]:
# Compute corpus and test model predictions
#corpus_predictions = model.forward(corpus_inputs).detach().round()
#test_predictions = model.forward(test_inputs).detach().round()
model.to('cpu')
corpus_predictions = model.forward(corpus_inputs)
sig = torch.nn.Sigmoid()
corpus_predictions = sig(corpus_predictions)

corpus_predictions, ub_corpus_predictions = new_threshold_fun(corpus_predictions)
corpus_predictions=torch.Tensor(corpus_predictions).unsqueeze(dim=1)
ub_corpus_predictions=torch.Tensor(ub_corpus_predictions).unsqueeze(dim=1)

Model_simplex_short(
  (layers): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=512, out_features=16, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
  )
  (layers2): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [27]:
# Fit SimplEx
# Compute the corpus and test latent representations
corpus_latents = model.latent_representation(corpus_inputs).detach()
test_latents = model.latent_representation(test_inputs).detach()
# Initialize SimplEX, fit it on test examples
simplex = Simplex(corpus_examples=corpus_inputs, corpus_latent_reps=corpus_latents)
simplex.fit(
    test_examples=test_inputs,
    n_epoch=100000,
    test_latent_reps=test_latents,
    reg_factor=0,
)

Weight Fitting Epoch: 20000/100000 ; Error: 0.0214 ; Regulator: 148 ; Reg Factor: 0
Weight Fitting Epoch: 40000/100000 ; Error: 0.00253 ; Regulator: 47.3 ; Reg Factor: 0
Weight Fitting Epoch: 60000/100000 ; Error: 0.00248 ; Regulator: 35.1 ; Reg Factor: 0
Weight Fitting Epoch: 80000/100000 ; Error: 0.00247 ; Regulator: 31 ; Reg Factor: 0
Weight Fitting Epoch: 100000/100000 ; Error: 0.00247 ; Regulator: 29.2 ; Reg Factor: 0


In [32]:
# Save SimplEx model
'''explainer_path = "simplex_hold_out_switch_model_short.pkl"
with open(explainer_path, "wb") as f:
    print(f"Saving SimplEx decomposition in {explainer_path}.")
    pkl.dump(simplex, f)'''

Saving SimplEx decomposition in simplex_hold_out_switch_model_short.pkl.


In [24]:
# Load SimplEx model
with open("simplex_hold_out_switch_model_short.pkl","rb") as file_handle:
    simplex = pickle.load(file_handle)

In [27]:
model.to('cpu')

Model_simplex_short(
  (layers): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=512, out_features=16, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
  )
  (layers2): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [28]:
# Compute the Integrated Jacobian for a particular example
i = 4
simplex.jacobian_projection(test_id=i, model=model, input_baseline=input_baseline)
result, sort_order = simplex.decompose(i, return_id=True)

tensor([[ 0.2602,  0.0546,  0.0112,  0.6174,  0.0032],
        [ 0.0341, -0.0344,  0.0048,  0.5352, -0.0049],
        [-0.0510, -0.0076, -0.0159,  0.2931, -0.0040],
        ...,
        [ 0.1143,  0.5633, -0.0286,  0.9692,  0.2162],
        [ 0.1411, -0.0790, -0.0124,  0.6508,  0.0115],
        [ 0.0051, -0.0609, -0.0173,  0.5424,  0.0021]])

In [29]:
len(result)

10015

In [30]:
corpus_dfs = [
    pd.DataFrame(
        result[j][1].numpy(),
        #columns=feature_names,
    )
    for j in range(len(result))]

In [31]:
corpus_data = [
    {
        "feature_vals": corpus_dfs[i].transpose(),
        "Label": apply_sort_order(corpus_targets.detach(), sort_order)[i],
        "Prediction": apply_sort_order(corpus_predictions.detach(), sort_order)[i],
        "ub_prediction": apply_sort_order(ub_corpus_predictions.detach(), sort_order)[i],
        "Example Importance": result[i][0],
    }
    for i in range(len(corpus_dfs))
]

In [32]:
corpus_data2 = [
    example
    for example in corpus_data
    if example["Example Importance"] >= 0.1
]

In [33]:
importance_dfs = [
    pd.DataFrame(
        result[j][2].numpy(),
        #columns=[f"{col}_fi" for col in feature_names],
    )
    for j in range(len(result))]

In [34]:
importance_data = [
    {
        "importance_vals": importance_dfs[i].transpose(),
        "Label": apply_sort_order(corpus_targets.detach(), sort_order)[i],
        "Prediction": apply_sort_order(corpus_predictions.detach(), sort_order)[i],
        "ub_prediction": apply_sort_order(ub_corpus_predictions.detach(), sort_order)[i],
        "Example Importance": result[i][0],
    }
    for i in range(len(corpus_dfs))
]

In [35]:
importance_data2 = [
    example
    for example in importance_data
    if example["Example Importance"] >= 0.1
]

In [36]:
# Clear as slows down loads
corpus_dfs = None
corpus_data = None
importance_dfs = None
importance_data = None

In [37]:
len(importance_data2)

4

In [16]:
import colorsys

def scale_lightness(hex_colour, scale_l):
    # Convert hex to rgb
    h = hex_colour.lstrip('#')
    rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    # convert rgb to hls
    rgb = [x/255.0 for x in rgb] # Divide by 255 as exspects numbers between 0 and 1
    h, l, s = colorsys.rgb_to_hls(*rgb)
    c = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb - i.e., Change brightness
    new_rgb = colorsys.hls_to_rgb(c[0], 1 - scale_l * (1 - c[1]), c[2])
    new_rgb = [x*255.0 for x in new_rgb]
    new_rgb = tuple(map(int, new_rgb))
    # convert back to hex
    new_hex = '#%02x%02x%02x' % new_rgb
    return new_hex

In [17]:
def df_values_to_colors(df):
    """Gets color values based in values relative to all other values in df."""

    min_val = np.nanmin(df.values)
    max_val = np.nanmax(df.values)
    for col in df:
        # map values to colors in hex via
        # creating a hex Look up table table and apply the normalized data to it
        norm = mcolors.Normalize(
            #vmin=min_val,
            #vmax=max_val,
            vmin=-1, # Changed to fixed max and min values fo colours cna be compared between examples
            vmax=1,
            # vmin=np.nanmin(df[col].values),
            # vmax=np.nanmax(df[col].values),
            clip=True,
        )
        lut = plt.cm.winter_r(np.linspace(0.2, 0.75, 256))
        lut = np.apply_along_axis(mcolors.to_hex, 1, lut)
        lut_shape = lut.shape
        lut_list = []
        for x in lut.flatten():
            new_lut = scale_lightness(x, 0.9)
            lut_list.append(new_lut)    
        lut = np.array(lut_list).reshape(lut_shape)
        a = (norm(df[col].values) * 255).astype(np.int16)
        df[col] = lut[a]
    return df


def highlight(x):
    return pd.DataFrame(importance_df_colors.values, index=x.index, columns=x.columns)

In [196]:
# Get colours of features
joint_feature_vals_colors = pd.DataFrame()
for x in range(len(corpus_data2)):
    example_i = x
    feature_vals_colors = df_values_to_colors(
        importance_data2[example_i]["importance_vals"].copy()
    )
    feature_vals_colors = feature_vals_colors.applymap(
        lambda x: f"background-color: {x}"
    )
    if x == 0:
        #master_df = display_corpus_df
        joint_feature_vals_colors = feature_vals_colors
    else:
        #master_df = pd.concat([master_df, display_corpus_df], axis=1)
        joint_feature_vals_colors = pd.concat([joint_feature_vals_colors, feature_vals_colors], axis=0)
joint_feature_vals_colors.index = np.arange(1, len(joint_feature_vals_colors) + 1)
joint_feature_vals_colors.columns = X_test_data.columns.to_list()

In [197]:
# Get colours of labels
joint_label_vals_colour = pd.DataFrame()
for example_dict in corpus_data2:
    working_df = pd.DataFrame({'switch_to_PO label': ['background-color: #none']})
    
    if example_dict["Prediction"][0].round() == example_dict["Label"]:
        if example_dict["ub_prediction"][0].round() == example_dict["Label"]:
        # importance_df_colors.applymap(lambda x: f"background-color: {x}"
        # joint_label_vals_colour
            working_df = pd.DataFrame({'switch_to_PO label': ['background-color: #none'], 'switch_to_PO prediction_threshold_1': ["background-color: #007a00"], 'switch_to_PO prediction_threshold_2': ["background-color: #007a00"]})
        else:
            working_df = pd.DataFrame({'switch_to_PO label': ['background-color: #none'], 'switch_to_PO prediction_threshold_1': ["background-color: #007a00"], 'switch_to_PO prediction_threshold_2': ["background-color: #7a0000"]})
        #joint_label_vals_colour = pd.concat([joint_label_vals_colour, working_df], axis=0)
    else:
        #working_df = pd.DataFrame({'switch_to_PO_prediction_threshold_1': ["background-color: #7a0000"]})
        #joint_label_vals_colour = pd.concat([joint_label_vals_colour, working_df], axis=0)
        if example_dict["ub_prediction"][0].round() == example_dict["Label"]:
        # importance_df_colors.applymap(lambda x: f"background-color: {x}"
        # joint_label_vals_colour
            working_df = pd.DataFrame({'switch_to_PO label': ['background-color: #none'], 'switch_to_PO prediction_threshold_1': ["background-color: #7a0000"], 'switch_to_PO prediction_threshold_2': ["background-color: #007a00"]})

            #working_df = pd.DataFrame({'switch_to_PO_prediction_threshold_2': ["background-color: #007a00"]})
            #joint_label_vals_colour = pd.concat([joint_label_vals_colour, working_df], axis=0)
        else:
            #working_df = pd.DataFrame({'switch_to_PO_prediction_threshold_2': ["background-color: #7a0000"]})
            working_df = pd.DataFrame({'switch_to_PO label': ['background-color: #none'], 'switch_to_PO prediction_threshold_1': ["background-color: #7a0000"], 'switch_to_PO prediction_threshold_2': ["background-color: #7a0000"]})

    joint_label_vals_colour = pd.concat([joint_label_vals_colour, working_df], axis=0)
joint_label_vals_colour.index = np.arange(1, len(joint_label_vals_colour) + 1)

In [198]:
# Get colour of importance
joint_importance_vals_colors = pd.DataFrame()
for example_dict in corpus_data2:
    # Get colour from map
    #cmap = mpl.cm.get_cmap('Greys')
    norm = mcolors.Normalize(vmin=-1, vmax=1, clip=True)
    #print(norm(example_dict["Example Importance"]))
    cmap = mpl.cm.get_cmap('Greys')
    rgba = cmap(norm(example_dict["Example Importance"]))
    #print(rgba)
    new_rgb = [x*255.0 for x in rgba]
    new_rgb = new_rgb[0:3]
    new_rgb = tuple(map(int, new_rgb))
    #print('new_rgb', new_rgb)
    # convert to hex
    new_hex = '#%02x%02x%02x' % new_rgb
    #print(new_hex)
    working_df = pd.DataFrame({'importance': [f"background-color: {new_hex}"]})
    joint_importance_vals_colors = pd.concat([joint_importance_vals_colors, working_df], axis=0)
joint_importance_vals_colors.index = np.arange(1, len(joint_importance_vals_colors) + 1)

In [199]:
# Join colours of importance and features and labels
master_colour_df = pd.concat([joint_importance_vals_colors, joint_feature_vals_colors, joint_label_vals_colour], axis=1)

In [200]:
# Create subject colour df
subject_colour_df = pd.DataFrame(index=np.arange(1), columns=np.arange(len(X_test_data.columns)+4))
subject_colour_df = subject_colour_df.fillna('background-color: #none')
subject_colour_df.columns = master_colour_df.columns

In [201]:
# Join colours of subject and exsampls
master_colour_df = pd.concat([subject_colour_df, master_colour_df], axis=0)

In [47]:
# Get features
joint_feature_vals = pd.DataFrame()
for example_dict in corpus_data2:
    #for keys, values in example_dict.items():
    joint_feature_vals = pd.concat([joint_feature_vals, example_dict["feature_vals"]], axis=0)

joint_feature_vals.index = np.arange(1, len(joint_feature_vals) + 1)
joint_feature_vals.columns = X_test_data.columns.to_list()

In [181]:
# Get labels
joint_label_vals = pd.DataFrame()
for example_dict in corpus_data2:
    working_df = pd.DataFrame({'switch_to_PO label': [example_dict["Label"]], 'switch_to_PO prediction_threshold_1': [example_dict["Prediction"][0].round().astype(int)], 'switch_to_PO prediction_threshold_2': [example_dict["ub_prediction"][0].round().astype(int)]})
    joint_label_vals = pd.concat([joint_label_vals, working_df], axis=0)
joint_label_vals.index = np.arange(1, len(joint_label_vals) + 1)

In [49]:
# Get importance
joint_importance_vals = pd.DataFrame()
for example_dict in corpus_data2:
    working_df = pd.DataFrame({'importance': [example_dict["Example Importance"]]})
    joint_importance_vals = pd.concat([joint_importance_vals, working_df], axis=0)
joint_importance_vals.index = np.arange(1, len(joint_importance_vals) + 1)

In [185]:
# Join importance and features and labels
joint_df = pd.concat([joint_importance_vals, joint_feature_vals, joint_label_vals], axis=1)
joint_df = joint_df.round(2)
joint_df['bool'] = 'example'
joint_df.reset_index(inplace=True)
# Convert to str for presentation
joint_df['index'] = joint_df['index'].astype(str)
joint_df['importance'] = joint_df['importance'].round(2).astype(str)

In [144]:
# Get predictions for test subject
model.to(device)
temp_test_dataset = MIMICDataset(test_data.iloc[[i]])
temp_test_dataloader = DataLoader(dataset=temp_test_dataset, batch_size=1, collate_fn=temp_test_dataset.collate_fn_padd)
temp_test_loss, temp_test_accuracy, temp_test_auroc, temp_test_predictions, temp_test_labels = evaluate(model, temp_test_dataloader, criterion)
temp_test_predictions, ub_temp_test_predictions = new_threshold_fun(temp_test_predictions)


Model_simplex_short(
  (layers): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=512, out_features=16, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
  )
  (layers2): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
  )
)

100%|██████████| 1/1 [00:00<00:00, 453.59it/s]


In [180]:
# Get subject data
subject_data = pd.DataFrame({'importance': ['']})
subject_data = pd.concat([subject_data, X_test_data.iloc[[i]].reset_index(drop=True)], axis=1)
subject_data = pd.concat([subject_data, test_data['po_flag'].iloc[[i]].reset_index(drop=True)], axis=1)
subject_data = subject_data.rename(columns={'po_flag': 'switch_to_PO label'})
subject_data['switch_to_PO prediction_threshold_1'] = temp_test_predictions
subject_data['switch_to_PO prediction_threshold_2'] = ub_temp_test_predictions
subject_data = subject_data.round(2)
subject_data['bool'] = 'patient'

In [186]:
# Join subject and subject dfs
joint_df2 = pd.concat([subject_data, joint_df], axis=0)
joint_df2 = joint_df2.fillna('')
joint_df2.set_index(['bool', 'index'], inplace=True)
joint_df2 = joint_df2.rename_axis([None, None]) # Remove names 
# Convert all to str for presentation
all_columns = list(joint_df2.columns[1:]) # Creates list of all column headers apart from importance
joint_df2[all_columns] = joint_df2[all_columns].round(2).astype(str)

In [194]:
joint_df2

importance blood pressure systolic17_current_stay heart rate17  \
patient                    0.44                                   0.59     
example 1       0.34       0.44                                   0.59     
        2       0.26       0.43                                    0.6     
        3       0.21       0.44                                   0.58     
        4       0.17       0.44                                   0.59     

          blood pressure mean11_current_stay  \
patient         0.22                           
example 1       0.23                           
        2       0.23                           
        3       0.17                           
        4       0.22                           

          o2 saturation pulseoxymetry3_current_stay  \
patient         0.46                                  
example 1       0.46                                  
        2       0.46                                  
        3       0.46                                  
        4       0.46                                  

          gcs - motor response15_current_stay switch_to_PO label  \
patient          0.5                                   0           
example 1       0.61                                   0           
        2       0.41                                   0           
        3       0.46                                   0           
        4       0.44                                   0           

          switch_to_PO prediction_threshold_1  \
patient            0                            
example 1          0                            
        2          0                            
        3          0                            
        4          0                            

          switch_to_PO prediction_threshold_2  
patient            0                           
example 1          0                           
        2          0                           
        3          0                           
        4          0

In [202]:
master_colour_df

,importance,blood pressure systolic17_current_stay,heart rate17,blood pressure mean11_current_stay,o2 saturation pulseoxymetry3_current_stay,gcs - motor response15_current_stay,switch_to_PO label,switch_to_PO prediction_threshold_1,switch_to_PO prediction_threshold_2
0,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...
1,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...
2,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...
3,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...
4,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...,backgr...


In [18]:
def highlight_master(x):
    return pd.DataFrame(master_colour_df.values, index=x.index, columns=x.columns)

In [203]:
display_df = joint_df2.style.apply(highlight_master, axis=None)

In [204]:
display_df

Final visualisations 

In [7]:
from functions_folder.simplex import *

GPU is available


In [8]:
# Load model
load_trained_model(model, 'VSCode_projects/stop/python_scripts/hold_out_switch_model_short.pt')

Model_simplex_short(
  (layers): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=512, out_features=16, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
  )
  (layers2): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [32]:
# Day 1
12311-12215
master_data_df, master_colour_df, display_df = simplex_df_fun(model, train_data, test_data, example=12311-12215)
display_df

96

100%|██████████| 1/1 [00:00<00:00, 546.20it/s]


In [36]:
# Day 2
12312-12215
master_data_df, master_colour_df, display_df = simplex_df_fun(model, train_data, test_data, example=12312-12215)
display_df

98

100%|██████████| 1/1 [00:00<00:00, 520.19it/s]


In [40]:
# Day 5
12315-12215
master_data_df, master_colour_df, display_df = simplex_df_fun(model, train_data, test_data, example=12315-12215)
display_df

100

100%|██████████| 1/1 [00:00<00:00, 525.01it/s]
